# Import Libraries

In [136]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import RegexTokenizer, CountVectorizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import IDF
from pyspark.ml import Pipeline, PipelineModel
import numpy as np
from pyspark.sql.functions import desc
from pyspark.sql.types import *
from pyspark.ml.feature import Word2Vec
from pyspark.sql.functions import col

In [137]:
spark = SparkSession \
        .builder \
        .appName("Content Based restaurant recommendation") \
        .getOrCreate()
sc = spark.sparkContext

# Load the sampled data

In [138]:
business_df = spark.read.parquet('Sample_Datasets/montreal_business')
reviews_df = spark.read.parquet('Sample_Datasets/montreal_reviews')
users_df = spark.read.parquet('Sample_Datasets/montreal_users')

# Process Reviews
1. Group reviews of restaurants
2. Generate a Word2Vec vector from the reviews.

### Step 1 : Group reviews of restaurants

In [168]:
#TODO new change
reviews_df.printSchema()
reviews_df_train, reviews_df_test = reviews_df.randomSplit([0.8, 0.2], seed=3)

reviews_text = reviews_df_train.select('business_id', 'text')
# Group reviews by restaurant ID
reviews_by_business = reviews_text.rdd.map(lambda x: (x[0], x[1])).reduceByKey(lambda x, y: x + " " + y)
reviews_by_business_df = reviews_by_business.toDF(['business_id', 'text'])
reviews_by_business_df.show(3)

root
 |-- business_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)



+--------------------+--------------------+
|         business_id|                text|
+--------------------+--------------------+
|-1xuC540Nycht_iWF...|I was so hungry f...|
|-4TVAE4KcLWVSXO2r...|These guys won't ...|
|-ldO91cdwbzIHN5hD...|Not a fan of this...|
+--------------------+--------------------+
only showing top 3 rows



In [169]:
from pyspark.sql.functions import avg

global_mean = reviews_df_train.agg(avg('stars')).collect()[0][0]
print("Global Average:",str(global_mean))

Global Average: 3.891145599712927


### Step 2: Convert reviews to Word2Vec feature vector

In [170]:
# Remove the stop words from text, and create the tf idf matrix
tokenize_sentence        = RegexTokenizer(pattern = '\w+', inputCol = 'text', outputCol = 'tokens', toLowercase=True, gaps = False)
remove_stopwords = StopWordsRemover(inputCol = 'tokens', outputCol = 'nostopwords')
# 1000 words to limit number of features and reduce overfitting
countVectorizer  = CountVectorizer(inputCol='nostopwords', outputCol='tf', vocabSize=1000)
tfiDF            = IDF(inputCol='tf', outputCol='tfidf_vec')
word2Vec = Word2Vec(vectorSize = 100, minCount = 5, inputCol = 'nostopwords', outputCol = 'word_2_vec', seed=123)
pipeline         = Pipeline(stages=[tokenize_sentence, remove_stopwords, countVectorizer, tfiDF, word2Vec])

reviews_pipeline = pipeline.fit(reviews_by_business_df)
reviews_pipeline.write().overwrite().save('reviews_pipeline')

In [171]:
reviews_pipeline = PipelineModel.load('reviews_pipeline')
transformed_reviews_by_business = reviews_pipeline.transform(reviews_by_business_df)
transformed_reviews_by_business.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- text: string (nullable = true)
 |-- tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- nostopwords: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- tf: vector (nullable = true)
 |-- tfidf_vec: vector (nullable = true)
 |-- word_2_vec: vector (nullable = true)



# Calculate cosine similarity between reviews of restaurants

In [172]:
def cosineSimilarity(vector1, vector2):
    '''
    This function calculates the cosine similarity between the restaurant feature vector and target restaurant feature vector
    using formula (A.B) / (sqrt((A**2)  * sqrt((B**2)))
    '''

    dot_product = np.dot(vector1, vector2)
    product_of_norms = np.sqrt(np.dot(vector1, vector1)) * np.sqrt(np.dot(vector2, vector2))

    return float(dot_product/product_of_norms)

In [173]:
def getUserRecommendationsTop10(user_restaurants, all_restaurants_word2vec) :
    '''
    This function uses cosine similarity to recommend restaurants based for the user
    '''

    # schema for output DF
    schema = StructType([
                            StructField("business_id", StringType(), True)
                            ,StructField("cosine_similarity", IntegerType(), True)
                            ,StructField("prediction", DoubleType(), True)
                        ])

    result_restaurants = spark.createDataFrame([], schema)

    for rest_id in user_restaurants:

        # Get the feature values i.e. review texts for the restaurants rated by the user
        user_rated_restaurant_features = all_restaurants_word2vec.filter(lambda x: x[0] == rest_id).map(lambda x: x[1]).first()

        # Calculate cosine similarity
        similar_restaurants_rdd = all_restaurants_word2vec.filter(lambda x: x[0] != rest_id).map(lambda x: (x[0], cosineSimilarity(x[1], user_rated_restaurant_features)))

        similar_restaurants_df = similar_restaurants_rdd.toDF(schema = ['business_id', 'cosine_similarity']).orderBy(desc('cosine_similarity'))

        # Scale cosine similarity to ratings 1-5
        similar_restaurants_df = similar_restaurants_df.withColumn("prediction", round((similar_restaurants_df['cosine_similarity'] - 0) / (1 - 0) * (5 - 1) + 1, 2))

        result_restaurants = result_restaurants.union(similar_restaurants_df)

    result_restaurants_top10 = result_restaurants.dropDuplicates(['business_id']).orderBy(desc('cosine_similarity'))

    return result_restaurants_top10

In [174]:
def getRestaurantDetails(sim_rest):
    '''
    Function to get the recommended restaurant details based on ids.
    '''

    restaurant_details = sim_rest.join(business_df, on='business_id', how = 'inner') \
                                 .select(sim_rest.business_id, \
                                       sim_rest.cosine_similarity, business_df.name, \
                                       business_df.categories, business_df.stars, business_df.review_count,
                                       business_df.latitude, business_df.longitude, sim_rest.prediction)

    return restaurant_details

# Find Best Recommendations

### 1. Best Restaurants based on past reviews of the user

In [147]:
train_users = reviews_df_train.select('user_id').distinct()
test_users = reviews_df_test.select('user_id').distinct()

common_users = train_users.join(test_users, ['user_id'], 'inner')
common_users.show(truncate=False)




+----------------------+
|user_id               |
+----------------------+
|EaBKe-8LB-NHuH7Us-QhGw|
|U9mfsF7Knvi4qkulS6k-Pg|
|VvBypfrIzspqwrbox_sUHA|
|M7vDDzoPNQDN2FdTcwCq4A|
|ucssLdHJmpo3lEyUtLi1Vg|
|6S4uxbPpb9pMk9bKMUVZng|
|eCy43X201JNHYKbPDN8xpA|
|s6qQj3otWtvBTv3wugeD5A|
|5cZEgqHwiN4XA2eOWWqnug|
|j78OeM6cLZ3OsFlJwwOgpA|
|4RjoIBBL450Z6EQenmWkSg|
|gurcmVDESVQ8X20xpw4btw|
|-7JSlmBJKUQwREG_yGuduQ|
|tCRK65LrfCEC1LzFS8LwvQ|
|IAP_k_esHa84vGKr8QerqQ|
|BxNBl3VrzLBAkwpJ3EUgXg|
|gGjQnTix1FeRPsOilTselA|
|xahq_WAFi6SYHFdwYVfVkg|
|B41w9LUYQHbkTjFRfpHEjA|
|N2HkEZn_EEtsy-KBg2NEmQ|
+----------------------+
only showing top 20 rows



In [182]:
from pyspark.sql.functions import rand

# Selecting a random user

# usr_id = reviews_df.select('user_id').orderBy(rand()).limit(1).collect()
# target_user = [val.user_id for val in usr_id][0]
#
# print(target_user)
# reviews_df.filter(reviews_df_train.user_id == target_user).show()

#has lot of reviews
# target_user = '5cZEgqHwiN4XA2eOWWqnug' 1.4 0.2
# target_user = '6S4uxbPpb9pMk9bKMUVZng' 0.91
# target_user = 'M7vDDzoPNQDN2FdTcwCq4A' 0.7
target_user = 'bds7VxgVm0e6Pu5RuVV-wg'
# target_user= 'VvBypfrIzspqwrbox_sUHA'
reviews_df_train.filter(reviews_df_train.user_id == target_user).show()
reviews_df_test.filter(reviews_df_test.user_id == target_user).show()
# target_user = 'KVehpTNCERwMGAx-h9MR-A'
#
#Create user profile from the reviews the user has given
user_reviews = reviews_df_train.filter( (reviews_df_train.user_id == target_user) & (reviews_df_train.stars >= float(3.0)) )\
                        .select(reviews_df_train.business_id).distinct()

#Here we use collect only to retrieve user reviewed restaurants, so it is safe
user_restaurants = [val.business_id for val in user_reviews.collect()]
user_past_restaurants = user_reviews.join(business_df, on='business_id', how = 'inner')

print(f'\nRestaurants reviewed by target user: {target_user}')
user_past_restaurants.select('business_id', 'name', 'categories', 'stars').show()

# (business_id => [reviews word vectors] )
all_restaurant_word2vec = transformed_reviews_by_business.select('business_id', 'word_2_vec') \
                                                    .rdd.map(lambda x: (x[0], x[1]))

#Fetch recommendations for target user
recommended_restaurants = getUserRecommendationsTop10(user_restaurants, all_restaurant_word2vec)


#Get details about the recommended restaurants
recommended_restaurants_details = getRestaurantDetails(recommended_restaurants)

print(f'\nRecommendations for User: {target_user}')
recommended_restaurants_details.drop('cosine_similarity','latitude', 'longitude') \
                           .orderBy(desc('cosine_similarity')).show(10)


+--------------------+--------------------+--------------------+-----+--------------------+
|         business_id|             user_id|           review_id|stars|                text|
+--------------------+--------------------+--------------------+-----+--------------------+
|-Levln3VcfeXS4iHt...|bds7VxgVm0e6Pu5Ru...|F0icbVOVtOhkdm8m1...|  4.0|We've been wantin...|
|3cSe8U3u-rscNruJ6...|bds7VxgVm0e6Pu5Ru...|mHxjvrB7h8bIvpUkV...|  3.0|Possibly one of t...|
|410NJLoLryd-vYNwR...|bds7VxgVm0e6Pu5Ru...|yqq3M_Pe-nBs15D33...|  4.0|I am giving this ...|
|9HWCSPtMP2tU_cTDs...|bds7VxgVm0e6Pu5Ru...|FnEwXtriuAmkqIcfd...|  4.0|I was so excited ...|
|BHTWG6p3FsnnlCGMo...|bds7VxgVm0e6Pu5Ru...|loqF3R55bMYJH8w0c...|  3.0|I wanted to come ...|
|EQZPTDAy-lWijb0i8...|bds7VxgVm0e6Pu5Ru...|qMML3TZEubADaxjPv...|  4.0|Fave dumpling pla...|
|EgRpbUZKPiotTXdrO...|bds7VxgVm0e6Pu5Ru...|q0dmEs3ibRHXEmjMl...|  5.0|I love their pad ...|
|FTJqVHCcnjkvM3Bva...|bds7VxgVm0e6Pu5Ru...|2Nqa4KyuwwFXNfpA4...|  4.0|Tried this

+--------------------+--------------------+--------------------+------------+--------------------+
|         business_id|             user_id|           review_id|actual_stars|                text|
+--------------------+--------------------+--------------------+------------+--------------------+
|O1P4x0RgJWb6Waetk...|bds7VxgVm0e6Pu5Ru...|j54bKcdUFvIr0CuQX...|         4.0|Great pizza!! The...|
|aeXwUDNI4e7NRQpRp...|bds7VxgVm0e6Pu5Ru...|F8yNPMqdCBOgPA5Xt...|         2.0|We came here on a...|
|oguosnOCL3M-LLx0p...|bds7VxgVm0e6Pu5Ru...|3x3pUTIGYlAm6XDqY...|         5.0|I came across thi...|
|xF4xbsAdhr44s9psc...|bds7VxgVm0e6Pu5Ru...|LfFoOS9k7wM9qWiFt...|         5.0|My favourite brea...|
+--------------------+--------------------+--------------------+------------+--------------------+




Restaurants reviewed by target user: bds7VxgVm0e6Pu5RuVV-wg
+--------------------+--------------------+--------------------+-----+
|         business_id|                name|          categories|stars|
+--------------------+--------------------+--------------------+-----+
|-Levln3VcfeXS4iHt...|                Nozy|Japanese, Restaur...|  4.5|
|QrFuG9RN3UfUeE843...|            Ramen-Ya|Noodles, Restaura...|  3.5|
|Jbj4VGceJVKXOoN0z...|   Dessert Cafe Momo|Coffee & Tea, Res...|  5.0|
|s0GeAnaUENWb2xAn_...|          Café Frida|Cafes, Mexican, B...|  4.5|
|q0GL761jc3ia_nEDt...|         Le Poké Bar|Seafood, Food, Ha...|  3.5|
|3cSe8U3u-rscNruJ6...|           Uniburger|Restaurants, Burg...|  4.0|
|qje0dtfyYDRrdgpmT...|   Pho Bang New York|Vietnamese, Resta...|  3.5|
|j3QIwGvwZHj459YHJ...|         Hotto Doggu|Hot Dogs, Restaur...|  3.5|
|s2I_Ni76bjJNK9yG6...|Maison Christian ...|Food, Sandwiches,...|  4.5|
|FTJqVHCcnjkvM3Bva...|           Seoul BBQ|Restaurants, Kore...|  4.0|
|nyeIcJRPs8n199n


Recommendations for User: bds7VxgVm0e6Pu5RuVV-wg


+--------------------+--------------------+--------------------+-----+------------+----------+
|         business_id|                name|          categories|stars|review_count|prediction|
+--------------------+--------------------+--------------------+-----+------------+----------+
|gZvwCOaMhxFXXNvy1...|             Marusan|Japanese, Restaur...|  4.5|         181|      4.93|
|xidr6_d3fwKSb_XaB...|            Le Ninja|Japanese, Sushi B...|  4.0|          32|      4.92|
|Wh_-S_tJvma15bxjS...|Saiko Bistrot Iza...|Sushi Bars, Japan...|  4.0|          78|      4.88|
|G1ynW1gclKMClO4It...|               Chops|Food, Asian Fusio...|  4.0|          15|      4.88|
|J2GTY3N_nH7d4L7ng...|     Espadon Plateau|Fish & Chips, Med...|  4.0|          13|      4.87|
|ZsA8J49XHGd6nrNhb...|              Toroli|Restaurants, Japa...|  4.5|          34|      4.86|
|EEHMMiJUklGGiikqd...|               Milos|  Restaurants, Greek|  4.0|         146|      4.86|
|AE-IzUqIkYfJrUqx3...|        La Republika|Filipin

# Evaluation

In [183]:
# target_user = 'bds7VxgVm0e6Pu5RuVV'

reviews_df_test = reviews_df_test.withColumnRenamed("stars", "actual_stars")
evaluation = recommended_restaurants_details.join(reviews_df_test.where((col('user_id') == target_user)),['business_id'],'inner')
evaluation.show()





+--------------------+------------------+-----------------+--------------------+-----+------------+----------+-----------+----------+--------------------+--------------------+------------+--------------------+
|         business_id| cosine_similarity|             name|          categories|stars|review_count|  latitude|  longitude|prediction|             user_id|           review_id|actual_stars|                text|
+--------------------+------------------+-----------------+--------------------+-----+------------+----------+-----------+----------+--------------------+--------------------+------------+--------------------+
|O1P4x0RgJWb6Waetk...|0.7782438667796558|            Adamo|  Restaurants, Pizza|  4.0|          55| 45.473375|  -73.58829|      4.11|bds7VxgVm0e6Pu5Ru...|j54bKcdUFvIr0CuQX...|         4.0|Great pizza!! The...|
|aeXwUDNI4e7NRQpRp...|0.7878070633128489|       C'ChoColat|Specialty Food, S...|  4.0|         203|45.4963458|-73.5755678|      4.15|bds7VxgVm0e6Pu5Ru...|F8yNPM

In [150]:
evaluation.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- cosine_similarity: double (nullable = true)
 |-- name: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- review_count: long (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- prediction: double (nullable = true)
 |-- user_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- actual_stars: double (nullable = true)
 |-- text: string (nullable = true)



In [184]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(metricName="rmse", labelCol="actual_stars",predictionCol="prediction")
rmse = evaluator.evaluate(evaluation)
print("rmse:",rmse)

rmse: 1.1721241401831122
